# Qubitization Walk Operator

In [ ]:
from qualtran import Bloq, CompositeBloq, BloqBuilder, Signature, Register
from qualtran import QBit, QInt, QUInt, QAny
from qualtran.drawing import show_bloq, show_call_graph, show_counts_sigma
from typing import *
import numpy as np
import sympy
import cirq

## `QubitizationWalkOperator`
Constructs a Szegedy Quantum Walk operator using LCU oracles SELECT and PREPARE.

Constructs a Szegedy quantum walk operator $W = R_{L} . \mathrm{SELECT}$, which is a product of
two reflections $R_{L} = (2|L\rangle \langle L| - I)$ and $\mathrm{SELECT} = \sum_{l}
|l\rangle\langle l|H_{l}$.

The action of $W$ partitions the Hilbert space into a direct sum of two-dimensional irreducible
vector spaces. For an arbitrary eigenstate $|k\rangle$ of $H$ with
eigenvalue $E_k$, $|\ell\rangle|k\rangle$ and
an orthogonal state $\phi_{k}$ span the irreducible two-dimensional space that
$|\ell\rangle|k\rangle$ is in under the action of $W$. In this space, $W$ implements a Pauli-Y
rotation by an angle of $-2\arccos(E_{k} / \lambda)$ s.t. $W = e^{i \arccos(E_k / \lambda) Y}$.

Thus, the walk operator $W$ encodes the spectrum of $H$ as a function of eigenphases of $W$
s.t. $spectrum(H) = \lambda cos(\arg(\mathrm{spectrum}(W)))$ where $\arg(e^{i\phi}) = \phi$.

#### Parameters
 - `select`: The SELECT lcu gate implementing $\mathrm{SELECT}=\sum_{l}|l\rangle\langle l|H_{l}$.
 - `prepare`: Then PREPARE lcu gate implementing $\mathrm{PREPARE}|00...00\rangle = \sum_{l=0}^{L - 1}\sqrt{\frac{w_{l}}{\lambda}} |l\rangle = |\ell\rangle$
 - `control_val`: If 0/1, a controlled version of the walk operator is constructed. Defaults to None, in which case the resulting walk operator is not controlled.
 - `power`: Constructs $W^{\mathrm{power}}$ by repeatedly decomposing into `power` copies of $W$. Defaults to 1. 

#### References
 - [Encoding Electronic Spectra in Quantum Circuits with Linear T Complexity](https://arxiv.org/abs/1805.03662). Babbush et. al. (2018). Figure 1.


In [ ]:
from qualtran.bloqs.qubitization_walk_operator import QubitizationWalkOperator

### Example Instances

In [ ]:
from qualtran.bloqs.qubitization_walk_operator_test import get_walk_operator_for_1d_ising_model

walk_op = get_walk_operator_for_1d_ising_model(4, 2e-1)

In [ ]:
from qualtran.bloqs.chemistry.sparse import PrepareSparse, SelectSparse
from qualtran.bloqs.chemistry.sparse.prepare_test import build_random_test_integrals

num_spin_orb = 8
num_bits_rot_aa = 8
num_bits_state_prep = 12
tpq, eris = build_random_test_integrals(num_spin_orb // 2)
prepare = PrepareSparse.from_hamiltonian_coeffs(
    num_spin_orb,
    tpq,
    eris,
    qroam_block_size=32,
    num_bits_state_prep=num_bits_state_prep,
    num_bits_rot_aa=num_bits_rot_aa,
)
select = SelectSparse(num_spin_orb)
walk_op_chem_sparse = QubitizationWalkOperator(select=select, prepare=prepare)

#### Graphical Signature

In [ ]:
from qualtran.drawing import show_bloqs
show_bloqs([walk_op, walk_op_chem_sparse],
           ['`walk_op`', '`walk_op_chem_sparse`'])

### Call Graph

In [ ]:
from qualtran.resource_counting.generalizers import ignore_split_join
walk_op_g, walk_op_sigma = walk_op_chem_sparse.call_graph(generalizer=ignore_split_join)
show_call_graph(walk_op_g)
show_counts_sigma(walk_op_sigma)